
# EDA Dataset de Violencia de Genero
## Analisis exploratorio de datos usando la libreria spaCy

Se identificaron en el dataset aquellas celdas con frases relevantes (celdas no vacias, que tengan frases distintas a 'no_corresponde' y que no esten repetidas). 
Para trabajar con el dataset xlsx se crearon un par de funciones en un modulo que se llamó xl_text que pasa las celdas con la informacion a textos que pueden ser procesados con spacy.

In [ ]:
# en archivo xl_text:

    
def excel_to_set(doc, first_cell, last_cell):
    """
    Esta funcion devuelve una lista con todas las frases unicas almacenadas en un grupo de celdas de un excel
    doc: string nombre del archivo excel a trabajar.
    first_cell: primera celda del rango a copiar.
    last_cell: ultima celda del rango a copiar
    """
    phrases = []
    excel_document = openpyxl.load_workbook(doc, data_only=True)
    first_sheet = excel_document.sheetnames[0]
    sheet = excel_document[first_sheet]
    for row in sheet[first_cell:last_cell]:
        for cell in row:
            if cell.value and cell.value != 'no_corresponde':  # si la celda tiene informacion
                cell_with_spaces = cell.value.replace('_', ' ')
                phrases.append(cell_with_spaces)
    return set(phrases)


def excel_to_text(doc, first_cell, last_cell):
    """ Esta funcion devuelve en forma de string los datos relevantes almacenados en un grupo de celdas de un excel
        doc: string nombre del archivo excel a trabajar.
        first_cell: primera celda del rango a copiar.
        last_cell: ultima celda del rango a copiar
        """
    phrases = excel_to_set(doc, first_cell, last_cell)
    text = ""
    for phrase in phrases:
        text += f'{phrase}\n'
    return text


def num_relevant_cells(doc, first_cell, last_cell):
    """Esta funcion escribe por pantalla estadisticas de el numero de celdas
    con informacion no nula en un rango de celdas de una columna en un excel

    doc: string nombre del archivo excel a trabajar.
    first_cell: primera celda del rango
    last_cell: ultima celda del rango
    """
    percentage = 0
    relevant_cells = len(excel_to_set(doc, first_cell, last_cell))
    total = int(last_cell[1:]) - int(first_cell[1:]) # solo funciona si trabajo en la misma columna. Sino usaria otro loop
    if total != 0:
        percentage = relevant_cells * 100 / total
    print(f'el numero total de celdas: {total} \n'
          f'el numero total de celdas con informacion unica: {relevant_cells}\n'
          f'eso es un %{percentage:.2f} de celdas con informacion')

En este caso se encontraron 337 celdas con frases unicas utiles para procesar: 

In [2]:
from xl_text import num_relevant_cells

num_relevant_cells('set_de_datos_con_perspectiva_de_genero.xlsx', first_cell='S2', last_cell='S3827')


el numero total de celdas: 3825 
el numero total de celdas con informacion unica: 337
eso es un %8.81 de celdas con informacion


   En primer lugar se analizaron estas frases encontradas en el dataset con el modelo preentrenado de spacy 'es_core_news_sm' (un modelo entrenado usando noticias). 
   - Se puede observar que este modelo logra indentificar de forma satisfactoria los part of speech tags (osea la categoria gramatical) y las dependencias (analisis sintactico de la relacion entre las palabras) de los tokens en cada frase.
   
   Con este primer procesamiento observamos las frases encontradas y analizamos la estructura de las mismas, lo que es beneficioso para lograr una mejor interpretación de los datos.  

In [1]:
import spacy
from xl_text import excel_to_set 

texts = list(excel_to_set('set_de_datos_con_perspectiva_de_genero.xlsx', first_cell='S2', last_cell='S3827'))
nlp = spacy.load("es_core_news_sm")
docs = [nlp(text) for text in texts]

print("tokens, su part-of-speech tag y dependency label:\n")
for doc in docs:
    print(f'\n{doc.text}')
    for token in doc:
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

tokens, su part-of-speech tag y dependency label:


hija de puta te mato te mato a vos a tu macho y a quien se meta me importa tres huevos la policia no me interesa ir preso
hija        NOUN      nsubj     
de          ADP       case      
puta        NOUN      nmod      
te          PRON      obj       
mato        VERB      nsubj     
te          PRON      obj       
mato        VERB      ROOT      
a           ADP       case      
vos         PROPN     obl       
a           ADP       case      
tu          DET       det       
macho       INTJ      obj       
y           CCONJ     cc        
a           ADP       case      
quien       PRON      obj       
se          PRON      obj       
meta        VERB      conj      
me          PRON      obj       
importa     VERB      advcl     
tres        NUM       nummod    
huevos      NOUN      nsubj     
la          DET       det       
policia     NOUN      nsubj     
no          ADV       advmod    
me          PRON      obj       
i

spacy tambien permite buscar las entidades presentes en el texto
   - En este caso observamos que en su mayoria no se logran identificar de forma satisfactoria entidades presentes. De todas formas no hay muchas entidades en las frases del dataset, ya que las frases consisten en su mayoria en insultos o amenazas que una persona le hace a otra, adicionalmente las frases del dataset se encuentran en su mayoria en minuscula lo que hace que al modelo se le dificulte identificar entidades como tal. 

In [26]:
print("entidades identificadas :\n")
for doc in docs:
    for ent in doc.ents:
        print(f'{ent.text} : {ent.label_}')

entidades identificadas :

aca : LOC
palos qué lloras : LOC
prender : PER
venis : PER
asi : ORG
servis : LOC
aca : LOC
No te hagas : MISC
Yo se donde vivís : MISC
trola    : MISC
gorda lechona : PER
mi palabra : MISC
la ley : LOC
mi casa yo hago : MISC
prender : PER
trola : MISC
aca : ORG
servis : LOC
mentirosa    : PER
zorra    : MISC
servis : LOC
ocurra : LOC
tenes : PER
servis : LOC
necesito : PER
meter : LOC
puta lesbiana : MISC
soy : ORG
puta lesbiana : MISC
palos : LOC
eh : ORG
palos : LOC
aca : LOC
boca : ORG
“Hola : PER
¿Querés : PER
“Este viejo pajero te : MISC
No camines sola : MISC
Dale : ORG
prender : PER
prender : PER
prender : PER
zorra    : MISC
meter : LOC
vivis aca : PER
zorra : MISC
TE AMO CACHIRULA : MISC
Este episodio : MISC
“ : MISC
Es menester hacer mención : MISC
“Facebook : MISC
puta sucia : ORG
bajate : LOC
qué me dejo : PER
está : PER
necesito : PER
soy : MISC
prender : PER
servis : LOC
soy : MISC
puta zorra : MISC
puta de mierda : MISC
vieja : LOC
andate : PE

Usando el PhraseMatcher de spacy podemos buscar instancias de palabras que pueden indicar amenazas o una frase amenazante. 
Con este podemos observar que las palabras "matar", "prender fuego" o "incendiar" aparecen 143 veces en las 337 frases del dataset. 
En este caso se busco en todas las frases como un texto entero, pero se puede aplicar de la misma forma a cada frase por separado .

In [3]:
from spacy.matcher import PhraseMatcher
from xl_text import excel_to_text

text = excel_to_text('set_de_datos_con_perspectiva_de_genero.xlsx', first_cell='S2', last_cell='S3827')
doc = nlp(text)
matcher = PhraseMatcher(nlp.vocab)
WORDS = ["matar", "prender fuego", "incendiar"]
patterns = [nlp(word) for word in WORDS]
matcher.add("AMENAZA_PATTERN", None, *patterns)
matches = matcher(doc)
threats = len(matches)
# matches contiene: match_id, start, end. Con eso puedo acceder al span que tiene el match. 
print(f'numero de apariciones de la palabra matar o similares: {threats}')

numero de apariciones de la palabra matar o similares: 143


Tambien se puede entrenar a un modelo de spacy para identificar esas palabras particulares como un nuevo tipo de entidad de texto que llamariamos 'amenaza'. Para esto creamos dos funciones get_training_data() y get_testing_data() que separan el dataset en datos de entrenamiento y de testeo. La data de entrenamiento es pasada a un conjunto de docs con las entidades 'amenaza' ya identificadas.
 - con estos datos entrenamos al modelo y observamos que identifica 113 entidades 'amenaza' en 186 frases.
 - si busco identificar si una frase es o no amenazante puedo suponer que si la frase tiene entidades amenazantes entonces lo es, en ese caso habria identificado 81 frases como amenazantes de las 186 analizadas. 


In [8]:
import warnings
import random
from spacy.matcher import Matcher
from xl_text import excel_to_set

def get_training_data():
    """ Esta funcion devuelve una lista de la forma [(frase ,{"entities": entities})...] 
        donde se encuentran 150 de las frases del dataset, y las entidades 'amenazas' presentes en ellas
    """
    nlp1 = spacy.load("es_core_news_sm")
    matcher = Matcher(nlp1.vocab)
    pattern_1 = [{"LEMMA": "matar"}]
    pattern_2 = [{"LEMMA": "incendiar"}]
    pattern_3 = [{"LEMMA": "pagar"}]
    pattern_4 = [{"LOWER": "fuego"}]
    matcher.add("AMENAZA", None, pattern_1, pattern_2, pattern_3, pattern_4)
    training_data = []
    txts = list(excel_to_set('set_de_datos_con_perspectiva_de_genero.xlsx', first_cell='S2', last_cell='S3827'))
    docs = [nlp1(text) for text in txts[:150]]
    for doc in docs:
        matches = matcher(doc)
        spans = [doc[start:end] for match_id, start, end in matches]
        entities = [(span.start_char, span.end_char, "AMENAZA") for span in spans]
        training_example = (doc.text, {"entities": entities})
        training_data.append(training_example)
    return training_data


def get_testing_data():
    """ Esta funcion devuelve una lista de frases del dataset desde la frase 150 en adelante """
    txts = list(excel_to_set('set_de_datos_con_perspectiva_de_genero.xlsx', first_cell='S2', last_cell='S3827'))
    return txts[150:]

nlp = spacy.load("es_core_news_sm")
ner = nlp.get_pipe("ner")
ner.add_label("AMENAZA")
TRAINING_DATA = get_training_data()
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]  
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]  
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():  
    warnings.filterwarnings("once", category=UserWarning, module='spacy') 
    for itn in range(10):
        random.shuffle(TRAINING_DATA)
        losses = {}
        # Crea lotes con los ejemplos e itera sobre ellos
        for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
            texts = [text for text, entities in batch]
            annotations = [entities for text, entities in batch]
            nlp.update(texts, annotations, losses=losses)
# path = Path('C:\Users\Usuario\Desktop\Practicas Profesionales\violencia de género\EDA\EDA_violencia_de_genero\projecto_eda')
# nlp.to_disk(path)

TESTING_DATA = get_testing_data()
docs = [nlp(text) for text in TESTING_DATA]
num_ent = 0
num_thr = 0
for doc in docs:
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    num_ent += len(doc.ents) 
    if  len(doc.ents) > 0 :
        num_thr += 1
    for token in doc:
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))
        
print(f"el numero de entidades encontradas en {len(TESTING_DATA)} frases fue: {num_ent} entidades")
print(f' numero de frases con entidades: {num_thr}')

Entities [('matar', 'AMENAZA'), ('matar', 'AMENAZA')]
soy         AUX       cop       
un          DET       det       
loco        NOUN      ROOT      
de          ADP       case      
mierda      NOUN      nmod      
te          PRON      obj       
voy         AUX       aux       
a           ADP       mark      
matar       VERB      csubj     
voy         AUX       aux       
a           ADP       mark      
pasar       VERB      xcomp     
en          ADP       case      
cana        NOUN      obl       
toda        DET       det       
mi          DET       det       
vida        NOUN      obj       
pero        CCONJ     cc        
a           ADP       case      
ese         DET       det       
hijo        NOUN      obj       
de          ADP       case      
puta        NOUN      nmod      
lo          PRON      obj       
voy         AUX       aux       
a           ADP       mark      
matar       VERB      conj      
hija        NOUN      obj       
de          ADP       

Para hacer un mejor analisis de las frases de agresion del dataset, podemos entrenar un nuevo componente en el pipeline de procesamiento llamado TextCategorizer, que identifica la naturaleza de un texto procesado. En este caso se entrenara el TextCategorizer para que identifique si una frase es o no es amenazante. Este dataset ya contiene una columa que indica la clasificacion de la frase, por lo que podemos usar el dataset para crear training data para entrenar al modelo.
- En este caso entrenamos un modelo en blanco pero se puede entrenar el TextCategorizer sobre un modelo preexistente.
- Por el bajo volumen de los datos para el entrenamiento usamos la arquitectura ensemble, que es la mas precisa pero es mas lenta que las demas opciones.
- tomamos 236 frases del dataset para el entrenamiento del TextCategorizer y las restantes para el testeo. 
- hacemos 25 iteraciones de entrenamiento, y para cada una de ellas mostramos precision, recall y f_score para monitoriar el entrenamiento del modelo. 


En el entrenamiento del modelo para crear la data de entrenamiento llamamos a esta funcion que fue creada en el modulo xl_text. La funcion crea las listas de frases y de categorias de cada una usando la categorizacion ya realizada en el excel  

In [14]:
import openpyxl

def excel_to_data(doc, first_cell_num, last_cell_num, col_phrases, col_cats):
    """ Esta funcion devuelve una lista con todas las frases unicas encontradas en un rango de celdas de un excel
    y una lista con las categorias amenazante, no amenazante para cada una de estas frases.
    doc: string nombre del archivo excel a trabajar.
    first_cell_num: numero de la primera celda del rango a copiar.
    last_cell_num:  numero de la ultima celda del rango a copiar
    col_phrases: columna donde estan las frases en el archivo doc
    col_cats: columna donde esta la clasificacion en el archivo doc
     """
    cat_list = []
    text_list = []
    excel_document = openpyxl.load_workbook(doc, data_only=True)
    first_sheet = excel_document.sheetnames[0]
    sheet = excel_document[first_sheet]
    for i in range(first_cell_num, last_cell_num):
        cell = sheet[col_phrases + str(i)]
        if cell.value and (cell.value != 'no corresponde') :
            text = cell.value.replace('_', ' ')
            if (text not in text_list):
                cat_cell = sheet[col_cats + str(i)]
                if cat_cell.value == 'amenazas':
                    cats = {'AMENAZANTE': 1, 'NO_AMENAZANTE': 0}
                else:
                    cats = {'AMENAZANTE': 0, 'NO_AMENAZANTE': 1}
                cat_list.append(cats)
                text_list.append(text)
    return text_list, cat_list

doc = "set_de_datos_con_perspectiva_de_genero.xlsx"
texts, cats = excel_to_data(doc, 2, 2500, 'S', 'H')
print (len(texts))

236


In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from xl_text import excel_to_data, excel_to_set
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}


"""model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
    init_tok2vec=("Pretrained tok2vec weights", "option", "t2v", Path),"""

model=None
output_dir=None
n_iter=25
init_tok2vec=None

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()

if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("es")
    print("Created blank 'es' model")

# add the text classifier to the pipeline if it doesn't exist
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe("textcat", config={"exclusive_classes": True, "architecture": "ensemble"})
    nlp.add_pipe(textcat, last=True)
else:
    textcat = nlp.get_pipe("textcat")

# add labels to text classifier
textcat.add_label("AMENAZANTE")
textcat.add_label("NO_AMENAZANTE")

doc = "set_de_datos_con_perspectiva_de_genero.xlsx"
texts, cats = excel_to_data(doc, 2, 2500, 'S', 'H')
train_texts = texts[:200]
dev_texts = texts[200:]
train_cats = cats[:200]
dev_cats = cats[200:]
train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))
print(f' Using {len(texts)} examples ({len(train_texts)} training, {len(dev_texts)} evaluation)')


# get names of other pipes to disable them during training
pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    if init_tok2vec is not None:
        with init_tok2vec.open("rb") as file_:
            textcat.model.tok2vec.from_bytes(file_.read())
    print("Entrenando el modelo...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    batch_sizes = compounding(4.0, 32.0, 1.001)
    for i in range(n_iter):
        losses = {}
        random.shuffle(train_data)
        batches = minibatch(train_data, size=batch_sizes)
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print(
            "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format( 
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
            )
        )

if output_dir is not None:
    with nlp.use_params(optimizer.averages):
        nlp.to_disk(output_dir)
    print(f"modelo guardado en:{output_dir}")


Created blank 'es' model
 Using 236 examples (200 training, 36 evaluation)
Entrenando el modelo...
LOSS 	  P  	  R  	  F  
1.425	0.444	0.444	0.444
1.176	0.667	0.667	0.667
0.960	0.694	0.694	0.694
0.855	0.694	0.694	0.694
0.437	0.667	0.667	0.667
0.345	0.611	0.611	0.611
0.297	0.611	0.611	0.611
0.208	0.667	0.667	0.667
0.185	0.611	0.611	0.611
0.087	0.611	0.611	0.611
0.089	0.583	0.583	0.583
0.053	0.639	0.639	0.639
0.065	0.639	0.639	0.639
0.042	0.639	0.639	0.639
0.034	0.611	0.611	0.611
0.023	0.611	0.611	0.611
0.036	0.639	0.639	0.639
0.031	0.611	0.611	0.611
0.027	0.583	0.583	0.583
0.017	0.583	0.583	0.583
0.026	0.583	0.583	0.583
0.013	0.611	0.611	0.611
0.013	0.611	0.611	0.611
0.010	0.611	0.611	0.611
0.013	0.611	0.611	0.611


Testeamos el modelo entrenado y se observa que se encontraron 96 frases amenazantes de las 147  y la mayoria de los resultados parecen correctos.

In [4]:
texts = excel_to_set('set_de_datos_con_perspectiva_de_genero.xlsx', first_cell='S2500', last_cell='S3827')
print(f'numero de frases a testear: {len(texts)}')
docs = [nlp(text) for text in texts]
num_amen = 0
for doc in docs:
    print(doc.text, doc.cats)
    if(doc.cats['AMENAZANTE'] > 0.5):
        num_amen += 1
print(f'numero de frases a amenazantes:{num_amen}')

numero de frases a testear: 147
te voy a perseguir te voy a hacer la vida imposibe {'AMENAZANTE': 0.996906578540802, 'NO_AMENAZANTE': 0.0030934219248592854}
yo mando aca hija de puta  estos son mis hijos varones y van a hacer lo que yo digo {'AMENAZANTE': 0.9846748113632202, 'NO_AMENAZANTE': 0.015325173735618591}
sino venis conmigo te vas a morir por coronavirus {'AMENAZANTE': 0.0027013213839381933, 'NO_AMENAZANTE': 0.9972986578941345}
queres quedarte con negros   no sabes quien soy yo {'AMENAZANTE': 0.006998348981142044, 'NO_AMENAZANTE': 0.9930016398429871}
puta todo vuelve si no estas conmigo no estas con nadie mejor que te
escondas. {'AMENAZANTE': 0.6886298656463623, 'NO_AMENAZANTE': 0.31137019395828247}
gorda lechona vas a vivir toda tu vida asi te falta mucho para bajar de peso {'AMENAZANTE': 0.2322021722793579, 'NO_AMENAZANTE': 0.7677978277206421}
hija de puta {'AMENAZANTE': 0.9332010746002197, 'NO_AMENAZANTE': 0.06679896265268326}
estas viva porque yo quiero hija de mil puta me 

Otra opcion es entrenar un modelo con la totalidad de las frases que se encuentran en el dataset, ya que todas estas ya se encuentran clasificadas, y guardar el modelo para utilizarlo con otros textos.